# **KoBERT 모델**

참고 코드: https://velog.io/@seolini43/KOBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8C%8C%EC%9D%B4%EC%8D%ACColab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 전처리

In [2]:
#train data 불러오기
import pandas as pd
filename = '/content/drive/MyDrive/인공지능프로그래밍/project/data/train5.csv'
data = pd.read_csv(filename)
data.head()

,talker,unethical text,intensity,chat,type,text
0,2,True,2.0,2,SEXUAL,특히 벗방보는 애들은 진짜 거세 시켜야함
1,3,True,2.0,3,SEXUAL,볼 게 없어서 벗방을 보냐 ㅋㅋㅋ
2,3,False,0.0,2,IMMORAL_NONE,쉴드가 아니라 국가가 면제해준거야.
3,1,True,2.0,1,SEXUAL,에로배우들을 데려다가 아이돌을 만드네
4,2,True,2.0,2,SEXUAL,쟤네들은 표정이 딱 야동인데..


In [13]:
####이진 분류의 경우의 라벨인코딩(욕설을 1, 나머지는 0)
data.loc[(data['type'] == "IMMORAL_NONE"), 'type'] = 0 #도덕, 무도덕 => 0
data.loc[(data['type'] == "DISCRIMINATION"), 'type'] = 0 #차별 => 0
data.loc[(data['type'] == "SEXUAL"), 'type'] = 0 #선정 => 0
data.loc[(data['type'] == "VIOLENCE"), 'type'] = 0 #폭력 => 0
data.loc[(data['type'] == "ABUSE"), 'type'] = 1 #욕설 => 1

In [ ]:
####다중 분류의 경우의 라벨인코딩
data.loc[(data['type'] == "IMMORAL_NONE"), 'type'] = 0 #도덕, 무도덕 => 0
data.loc[(data['type'] == "DISCRIMINATION"), 'type'] = 1 #차별 => 1
data.loc[(data['type'] == "SEXUAL"), 'type'] = 2 #선정 => 2
data.loc[(data['type'] == "VIOLENCE"), 'type'] = 3 #폭력 => 3
data.loc[(data['type'] == "ABUSE"), 'type'] = 4 #욕설 => 4

In [14]:
train_data = data[['text','type']].to_numpy()
train_data

array([['특히 벗방보는 애들은 진짜 거세 시켜야함', 0],
       ['볼 게 없어서 벗방을 보냐 ㅋㅋㅋ', 0],
       ['쉴드가 아니라 국가가 면제해준거야.', 0],
       ...,
       ['누가 안 뺏어먹으니까 추접스럽게 후루룩거리지 말라고 걸뱅아', 1],
       ['진짜 운전 개좉같이 하네', 1],
       ['아 진짜 뚝배기 개 깨고 싶네', 1]], dtype=object)

In [15]:
#val data 불러오기
filename2 = '/content/drive/MyDrive/인공지능프로그래밍/project/data/val5.csv'
data2 = pd.read_csv(filename2)
data2.head()

,talker,unethical text,intensity,chat,type,text
0,1,False,0.0,1,IMMORAL_NONE,내 남동생은 키가 왜이렇게 안크지?
1,2,False,0.0,1,IMMORAL_NONE,지금이 딱 성장기라 클 땐데 ...
2,1,False,0.0,1,IMMORAL_NONE,근데 학교 폭력은 피해자 한테도 잘못이 있는 것 같아.
3,4,False,0.0,1,IMMORAL_NONE,부모님 치매 걸리면 어떻게 살아?
4,9,False,0.0,2,IMMORAL_NONE,그거 너무 차별적인 것 같은데.


In [16]:
####이진 분류일 경우의 라벨인코딩(욕설을 1, 나머지는 0)
data2.loc[(data2['type'] == "IMMORAL_NONE"), 'type'] = 0 #도덕, 무도덕 => 0
data2.loc[(data2['type'] == "DISCRIMINATION"), 'type'] = 0 #차별 => 0
data2.loc[(data2['type'] == "SEXUAL"), 'type'] = 0 #선정 => 0
data2.loc[(data2['type'] == "VIOLENCE"), 'type'] = 0 #폭력 => 0
data2.loc[(data2['type'] == "ABUSE"), 'type'] = 1 #욕설 => 1

In [ ]:
####다중 분류일 경우의 라벨인코딩
data2.loc[(data2['type'] == "IMMORAL_NONE"), 'type'] = 0 #도덕, 무도덕 => 0
data2.loc[(data2['type'] == "DISCRIMINATION"), 'type'] = 1 #차별 => 1
data2.loc[(data2['type'] == "SEXUAL"), 'type'] = 2 #선정 => 2
data2.loc[(data2['type'] == "VIOLENCE"), 'type'] = 3 #폭력 => 3
data2.loc[(data2['type'] == "ABUSE"), 'type'] = 4 #욕설 => 4

In [18]:
val_data = data2[['text','type']].to_numpy()
val_data

array([['내 남동생은 키가 왜이렇게 안크지?', 0],
       ['지금이 딱 성장기라 클 땐데 ...', 0],
       ['근데 학교 폭력은 피해자 한테도 잘못이 있는 것 같아.', 0],
       ...,
       ['남자 아래에 깔려서 당하고 싶은건가?', 0],
       ['저정도면 놀아달라는 거 아닌가? 저 라인봐봐 만지면 으스러지겠어', 0],
       ['저 언덕 사이에 손을 넣고 주물러주면 그럴걸 ㅋㅋ 본인도 바라고 있지 않을까?', 0]], dtype=object)

# 학습 모델

In [3]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 31.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp38-cp38-linux_x86_64.whl size=619635 sha256=d55015ecaa074c14bfbba5d133a2e56de921edd7f67f8da19be07e8c5cf9b438
  Stored in directory: /root/.cache/pip/wheels/b6/93/9d/2237550c409eb3ed725d6302b7897ddd9a037b40cef66dcd9c
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 28.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pk

In [4]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-0q3vd4zc
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-0q3vd4zc
     |████████████████████████████████| 129 kB 22.0 MB/s 
     |████████████████████████████████| 54.7 MB 37 kB/s 
     |████████████████████████████████| 4.5 MB 16.5 MB/s 
     |████████████████████████████████| 1.2 MB 44.1 MB/s 
     |██████████████████████████████▎ | 834.1 MB 11.8 MB/s eta 0:00:05tcmalloc: large alloc 1147494400 bytes == 0x39dae000 @  0x7f466f89c615 0x5d631c 0x51e4f1 0x51e67b 0x4f7585 0x49ca7c 0x4fdff5 0x49caa1 0x4fdff5 0x49ced5 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x5d7c18 0x5d9412 0x586636 0x5d813c 0x55f3fd 0x55e571 0x5d7cf1 0x49ced5 0x55e571 0x5d7cf1 0x49ec69 0x5d7c18 0x49ca7c 0x4fdff5 0x49ced5
     |████████████████████████████████| 881.9 MB

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [6]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [7]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
##GPU 사용 시
device = torch.device("cuda:0")

In [9]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [10]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [11]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 7
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [19]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train_data, 0, 1, tok, max_len, True, False)
data_val = BERTDataset(val_data, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_val, batch_size=batch_size, num_workers=5)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [20]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=8,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-24-480b6a139979>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1781 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.070470094680786 train acc 0.09375
epoch 1 batch id 201 loss 0.19065360724925995 train acc 0.7503109452736318
epoch 1 batch id 401 loss 0.06795904785394669 train acc 0.855556421446384
epoch 1 batch id 601 loss 0.30238839983940125 train acc 0.8830334858569051
epoch 1 batch id 801 loss 0.3009798526763916 train acc 0.8861579275905118
epoch 1 batch id 1001 loss 0.47892361879348755 train acc 0.8778721278721279
epoch 1 batch id 1201 loss 0.25452178716659546 train acc 0.8724890716069942
epoch 1 batch id 1401 loss 0.5315055847167969 train acc 0.8700816381156317
epoch 1 batch id 1601 loss 0.41306906938552856 train acc 0.86933947532792
epoch 1 train acc 0.8672708450308815


<ipython-input-24-480b6a139979>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/198 [00:00<?, ?it/s]

...

epoch 2 batch id 1 loss 0.3268911838531494 train acc 0.890625
epoch 2 batch id 201 loss 0.12406887114048004 train acc 0.962764303482587
epoch 2 batch id 401 loss 0.05481500178575516 train acc 0.9642300498753117
epoch 2 batch id 601 loss 0.11435074359178543 train acc 0.9601705490848585
epoch 2 batch id 801 loss 0.30202579498291016 train acc 0.9484043383270911
epoch 2 batch id 1001 loss 0.4257228970527649 train acc 0.9339410589410589
epoch 2 batch id 1201 loss 0.16579824686050415 train acc 0.9245680682764363
epoch 2 batch id 1401 loss 0.4135432541370392 train acc 0.9188637580299786
epoch 2 batch id 1601 loss 0.3810463547706604 train acc 0.915336118051218
epoch 2 train acc 0.9121593506858106


  0%|          | 0/198 [00:00<?, ?it/s]

epoch 2 test acc 0.8786314349636718


  0%|          | 0/1781 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.19850486516952515 train acc 0.96875
epoch 3 batch id 201 loss 0.045686084777116776 train acc 0.9735696517412935
epoch 3 batch id 401 loss 0.05880575627088547 train acc 0.97330891521197
epoch 3 batch id 601 loss 0.11828985810279846 train acc 0.9702059068219634
epoch 3 batch id 801 loss 0.2926866412162781 train acc 0.9607326779026217
epoch 3 batch id 1001 loss 0.2812272906303406 train acc 0.9490977772227772
epoch 3 batch id 1201 loss 0.1226050853729248 train acc 0.9410126977518735
epoch 3 batch id 1401 loss 0.3320595622062683 train acc 0.9358382405424697


# 사용자가 입력하는 문장의 클래스 예측

In [21]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("욕설이 포함되지 않은")
            elif np.argmax(logits) == 1:
                test_eval.append("욕설이 포함된")

        print(">> 입력하신 내용은 " + test_eval[0] + " 내용입니다.")

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
#질문 무한반복하기! (0일때 멈춤)
end = 1
while end == 1 :
    sentence = input("말을 입력해주세요 : ")
    if sentence == '0' :
        break
    predict(sentence)
    print("\n")